In [2]:
import numpy as np
import cv2
import math
import sys
import timeit

import matplotlib.pyplot as plt
%config IPCompleter.greedy=True # test autocompletion with tab or tab+shift

############################################################
#
#                       KMEANS
#
############################################################

# implement distance metric - e.g. squared distances between pixels
def distance(a, b):
    # YOUR CODE HERE
    return np.linalg.norm(a-b)

# k-means works in 3 steps
# 1. initialize
# 2. assign each data element to current mean (cluster center)
# 3. update mean
# then iterate between 2 and 3 until convergence, i.e. until ~smaller than 5% change rate in the error

def update_mean(img, clustermask):
    """This function should compute the new cluster center, i.e. numcluster mean colors"""
    
    unique, counts = np.unique(clustermask, return_counts=True)# YOUR CODE HERE
    #print('unique:',unique, 'counts:', counts )
    
    #tmp = clustermask==0
    #redimg1 = img*tmp
    #tmp = clustermask==1
    #redimg2 = img*tmp
    #tmp = clustermask==2
    #redimg3 = img*tmp
    #cv2.imshow("test1", redimg1)
    #cv2.imshow("test2", redimg2)
    #cv2.imshow("test3", redimg3)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    
    #print('Old:',current_cluster_centers)
    
    for k in range(numclusters):
        tmp = clustermask==k
        redimg = img*tmp
        for i in range(current_cluster_centers.shape[1]):
            current_cluster_centers[k,i] = np.sum(redimg[:,:,i]) / counts[k]
            
    #print('New:',current_cluster_centers)
    pass
        
def assign_to_current_mean(img, clustermask):
    """The function expects the img, the resulting image and a clustermask.
    After each call the pixels in result should contain a cluster_color corresponding to the cluster
    it is assigned to. clustermask contains the cluster id (int [0...num_clusters]
    Return: the overall error (distance) for all pixels to there closest cluster center (mindistance px -> cluster center).
    """
    overall_dist = 0
    # YOUR CODE HERE
    tmp = np.zeros(numclusters,dtype=float)
    #print(tmp)
    for r in range(h1):
        for c in range(w1):
            for k in range(numclusters):
                tmp[k] = distance(current_cluster_centers[k,:], img[r,c,:])
                
            idx = np.argmin(tmp)
            clustermask[r,c] = idx
            overall_dist += np.min(tmp)

    return overall_dist

def initialize(img):
    """inittialize the current_cluster_centers array for each cluster with a random pixel position"""
    # YOUR CODE HERE
    # random numclusters from pixels
    
    # numclusters
    xcl = np.random.randint(low=0, high=w1-1, size=numclusters)
    ycl = np.random.randint(low=0, high=h1-1, size=numclusters)
     
    #print('width x {} height y {}'.format(xcl,ycl))
    #randpos = np.vstack([ycl,xcl]).T
    #print(randpos)
    #print(current_cluster_centers.shape[0])
    
    for i in range(numclusters):
        current_cluster_centers[i,:]  = img[ycl[i],xcl[i],:]
    
    return current_cluster_centers


def kmeans(img, cflag):
    """Main k-means function iterating over max_iterations and stopping if
    the error rate of change is less then 2% for consecutive iterations, i.e. the
    algorithm converges. In our case the overall error might go up and down a little
    since there is no guarantee we find a global minimum.
    """
    max_iter = 10
    max_change_rate = 0.02
    dist_old = sys.float_info.max
    dist_new = 0

    clustermask = np.zeros((h1, w1, 1), np.uint8)
    result = np.zeros((h1, w1, 3), np.uint8)

    # initializes each pixel to a cluster
    # iterate for a given number of iterations or if rate of change is
    # very small
    # YOUR CODE HERE
    # init 1)
    current_cluster_centers = initialize(img)
    #print(current_cluster_centers.shape)
    
    # iteration
    while max_iter>0:
        # sort pixels to ks
        dist_new = assign_to_current_mean(img, clustermask)
        
        percent = (dist_old - dist_new) / dist_old
        print('Error: {0:.3f}'.format(percent))
        
        # display plot in notebook
        #%matplotlib inline
        #plt.figure(figsize=(6,6))
        #gbr = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
        #plt.imshow(gbr)
        #plt.show()
        
        if percent<max_change_rate:
            # here paint paint final result once
            for k in range(numclusters):
                for r in range(h1):
                    for c in range(w1):
                        if cflag==0: # cflag 0: use demo colors
                            result[r,c,:] = cluster_colors[int(clustermask[r,c,0])]
                        else: # cflag 1: use color from k center point = img pixels
                            result[r,c,:] = current_cluster_centers[int(clustermask[r,c,0])]
            break
            
        dist_old = dist_new  
        
        # review center of means
        update_mean(img, clustermask)
        max_iter-=1
    
    return result

# num of cluster
numclusters = 5
# corresponding colors for each cluster
cluster_colors = [[255, 0, 0], [0, 255, 0], [0, 0, 255], [0, 255, 255], [255, 255, 255], [0, 0, 0], [128, 128, 128]]
# initialize current cluster centers (i.e. the pixels that represent a cluster center)
current_cluster_centers = np.zeros((numclusters, 3), np.float32)
#print(current_cluster_centers.shape)

# load image
#imgraw = cv2.imread('Lenna.png')
imgraw = cv2.imread('neuro.png')
#imgraw = cv2.imread('nature.png')
scaling_factor = 0.5
imgraw = cv2.resize(imgraw, None, fx=scaling_factor, fy=scaling_factor, interpolation=cv2.INTER_AREA)

# compare different color spaces and their result for clustering
# YOUR CODE HERE or keep going with loaded RGB colorspace img = imgraw

# execute k-means over the image
# it returns a result image where each pixel is color with one of the cluster_colors
# depending on its cluster assignment
cflag = 0 # 0: use cluster_color or 1: center image color
h1, w1 = imgraw.shape[:2]
#print(h1,w1)
print('Running kmeans for two color spaces and RGB')
vis = np.zeros((3*h1, 2*w1, 3), np.uint8)
for n in range(3):
    if n == 0:
        #HSV
        print('kmean for HSV for 5 clusters')
        image = cv2.cvtColor(imgraw, cv2.COLOR_BGR2HSV)
        cflag = 0
    elif n==1:
        #YUV
        print('kmean for YUV for 5 clusters')
        image = cv2.cvtColor(imgraw, cv2.COLOR_BGR2YUV)
        cflag = 0
    else:
        #orig color image
        print('kmean for RGB for 5 clusters (use image color)')
        image = imgraw
        cflag = 1
    
    res = kmeans(image, cflag)
    
    vis[h1*n:h1*(n+1), :w1] = res
    vis[h1*n:h1*(n+1), w1:(2*w1)] = image

cv2.imshow("Color-based Segmentation Kmeans-Clustering [HSV,YUV,RGB]", vis)
cv2.waitKey(0)
cv2.destroyAllWindows()

#cv2.imwrite('res.png',vis)

Running kmeans for two color spaces and RGB
kmean for HSV for 5 clusters
Error: 1.000
Error: 0.264
Error: 0.122
Error: 0.021
Error: 0.009
kmean for YUV for 5 clusters
Error: 1.000
Error: 0.201
Error: 0.037
Error: 0.014
kmean for RGB for 5 clusters (use image color)
Error: 1.000
Error: 0.244
Error: 0.052
Error: 0.008


True